In [ ]:
#!/usr/bin/env python
# -*- encoding: utf-8 -*-
# Created on 2016-11-10 23:45:22
# Project: wanjiebook

from pyspider.libs.base_handler import *

import urllib
DIR = "files/"

class Handler(BaseHandler):
    crawl_config = {
    }
    
    file = open(DIR + 'books.txt','w')

    @every(minutes=24 * 60)
    def on_start(self):
        self.crawl('http://www.ebuk.cn/quanben.html', callback=self.index_page)
        

    @config(age=10 * 24 * 60 * 60)
    def index_page(self, response):
        
        for each in response.doc('.t > a').items():
            self.crawl(each.attr.href, callback=self.detail_page)
            
        for each in response.doc('#next_page').items():
            self.crawl(each.attr.href, callback=self.index_page)
            
        self.file.close()

    @config(priority=2)
    def detail_page(self, response):
        title = DIR + response.doc('h2 a').text() + '.txt'
        url = response.doc('.box1 .txt a').attr.href
        
        #urllib.urlretrieve(url, title)
        json = {"title": title, "url": url}
        
        self.file.write(url + '\n')
        
        return json

In [ ]:
#!/usr/bin/env python
# -*- encoding: utf-8 -*-
# Created on 2016-11-10 23:23:37
# Project: longbuluo

from pyspider.libs.base_handler import *


class Handler(BaseHandler):
    crawl_config = {
    }

    @every(minutes=24 * 60)
    def on_start(self):
        self.crawl('http://www.lbldy.com/movie/', callback=self.index_page)

    @config(age=10 * 24 * 60 * 60)
    def index_page(self, response):
        for each in response.doc('a[href^="http"]').items():
            if each.attr.href.endswith('html'):
                self.crawl(each.attr.href, callback=self.detail_page)
                
        for each in response.doc('a.next').items():
            self.crawl(each.attr.href, callback=self.index_page)

    @config(priority=2)
    def detail_page(self, response):
        title = response.doc('.col h2').text()
        elements = response.doc('div.entry > p > a').items()
        for each in elements:
            url =  each.attr.href
            if url.startswith('thunder'):
                print(url)
            else:
                print("Droped!")
                